In [2]:
import scipy.io as sio
import numpy as np

dataReadyForCNN = sio.loadmat("DataForCNN.mat")

xTest = dataReadyForCNN["xTest"]
xTrain = dataReadyForCNN["xTrain"]

yTrainCond = dataReadyForCNN["yTrain"]
yTestCond = dataReadyForCNN["yTest"]
#yTrainWord = dataReadyForCNN["yTrainWord"]
#yTestWord = dataReadyForCNN["yTestWord"]

In [3]:
xTest.shape



(648, 1, 51, 61, 23)

In [4]:
import theano
import theano.tensor as T
xTest_knn=xTest.reshape((xTest.shape[0],np.prod(xTest.shape[1:])))
xTrain_knn=xTrain.reshape((xTrain.shape[0],np.prod(xTrain.shape[1:])))

#sio.savemat("knnData.mat",{"xTest_knn":xTest_knn, "xTrain_knn":xTrain_knn, "yTrainCond_knn":yTrainCond, "yTestCond_knn":yTestCond,"yTrainWord_knn":yTrainWord, "yTestWord_knn":yTestWord})


In [5]:
xTrain_knn.shape

(2592, 71553)

In [166]:
import scipy.io as sio
import numpy as np
import theano.tensor as T
import theano
from theano import shared
from lasagne.layers import InputLayer, DenseLayer

from mlp import LogRegr, HiddenLayer, DropoutLayer
from convnet3d import ConvLayer, NormLayer, PoolLayer, RectLayer
from activations import relu, tanh, sigmoid, softplus

# xTrain = np.random.rand(500, 1, 51, 61, 23).astype('float64')

dtensor5 = T.TensorType('float64', (False,)*5)
x = dtensor5('x') # the input data
y = T.ivector()

# input = (nImages, nChannel(nFeatureMaps), nDim1, nDim2, nDim3)

# layer1 (500, 5, 47, 56, 22)
# layer2 (500, 5, 10, 12, 5)
# layer3 (500, 3, 9, 11, 4)
# layer4 (500, 3, 5, 6, 2)

kernel_shape = (5,6,2)
fMRI_shape = (51, 61, 23)
n_in_maps = 1 # channel
n_out_maps = 5 # num of feature maps, aka the depth of the neurons
batch_size = 500

# 1st: Convolution Layer
layer1_input = x
layer1 = ConvLayer(layer1_input, 1, 5, (5, 6, 2), fMRI_shape, 
                       batch_size, tanh)

# print layer1.output.eval({x:xTrain[:500]}).shape

# 2nd: Pool layer
poolShape = (5, 5, 5)
layer2 = PoolLayer(layer1.output, poolShape)

# print layer2.output.eval({x:xTrain}).shape

# 3rd: Convolution Layer
layer3 = ConvLayer(layer2.output, 5, 3, (2, 2, 2), (10, 12, 5), 
                       500, tanh)

# print layer3.output.eval({x:xTrain[:500]}).shape

# 4th: Pool layer
layer4 = PoolLayer(layer3.output, (2, 2, 2))

# print layer4.output.eval({x:xTrain[:500]}).shape

# 5th: Dense layer
layer5_input = T.flatten(layer4.output, outdim=2)
layer5 = HiddenLayer(layer5_input, n_in=180, n_out=500, activation=tanh)

# layer5.output.eval({x:xTrain[:500]}).shape

# 6th: Logistic layer
layer6 = LogRegr(layer5.output, 500, 12, tanh)

cost = layer6.negative_log_likelihood(y)

# # create a function to compute the mistakes that are made by the model
# test_model = theano.function(
#     [index],
#     layer6.errors(y),
#     givens={
#         x: test_set_x[index * batch_size: (index + 1) * batch_size],
#         y: test_set_y[index * batch_size: (index + 1) * batch_size]
#     }
# )

# validate_model = theano.function(
#     [index],
#     layer6.errors(yCond),
#     givens={
#         x: valid_set_x[index * batch_size: (index + 1) * batch_size],
#         y: valid_set_y[index * batch_size: (index + 1) * batch_size]
#     }
# )

# create a list of all model parameters to be fit by gradient descent
params = layer5.params + layer3.params + layer1.params + layer6.params

# create a list of gradients for all model parameters
grads = T.grad(cost, params)

# train_model is a function that updates the model parameters by
# SGD Since this model has many parameters, it would be tedious to
# manually create an update rule for each model parameter. We thus
# create the updates list by automatically looping over all
# (params[i], grads[i]) pairs.
learning_rate=0.1

updates = [
    (param_i, param_i - learning_rate * grad_i)
    for param_i, grad_i in zip(params, grads)
]

# allocate symbolic variables for the data
index = T.lscalar()  # index to a [mini]batch

train_model = theano.function(
    [index],
    cost,
    updates=updates,
    givens={
        x: shared(xTrain)[index * batch_size: (index + 1) * batch_size],
        y: shared(yTrainCond)[index * batch_size: (index + 1) * batch_size]
    }
)

In [85]:
xTrain[1:5].shape

(4, 1, 51, 61, 23)

In [167]:
###############
# TRAIN MODEL #
###############
import timeit

print('... training')

n_train_batches = 3
n_epochs=200

# early-stopping parameters
patience = 10000  # look as this many examples regardless
patience_increase = 2  # wait this much longer when a new best is
                       # found
improvement_threshold = 0.995  # a relative improvement of this much is
                               # considered significant
validation_frequency = min(n_train_batches, patience // 2)
                              # go through this many
                              # minibatche before checking the network
                              # on the validation set; in this case we
                              # check every epoch

best_validation_loss = np.inf
best_iter = 0
test_score = 0.
start_time = timeit.default_timer()

epoch = 0
done_looping = False

while (epoch < n_epochs) and (not done_looping):
    epoch = epoch + 1
    for minibatch_index in range(n_train_batches):

        minibatch_avg_cost = train_model(minibatch_index)
        # iteration number
        iter = (epoch - 1) * n_train_batches + minibatch_index

        if (iter + 1) % validation_frequency == 0:
            # compute zero-one loss on validation set
            validation_losses = [validate_model(i) for i
                                 in range(n_valid_batches)]
            this_validation_loss = numpy.mean(validation_losses)

            print(
                'epoch %i, minibatch %i/%i, validation error %f %%' %
                (
                    epoch,
                    minibatch_index + 1,
                    n_train_batches,
                    this_validation_loss * 100.
                )
            )

            # if we got the best validation score until now
            if this_validation_loss < best_validation_loss:
                #improve patience if loss improvement is good enough
                if (
                    this_validation_loss < best_validation_loss *
                    improvement_threshold
                ):
                    patience = max(patience, iter * patience_increase)

                best_validation_loss = this_validation_loss
                best_iter = iter

                # test it on the test set
                test_losses = [test_model(i) for i
                               in range(n_test_batches)]
                test_score = numpy.mean(test_losses)

                print(('     epoch %i, minibatch %i/%i, test error of '
                       'best model %f %%') %
                      (epoch, minibatch_index + 1, n_train_batches,
                       test_score * 100.))

        if patience <= iter:
            done_looping = True
            break

            
# end_time = timeit.default_timer()
# print(('Optimization complete. Best validation score of %f %% '
#        'obtained at iteration %i, with test performance %f %%') %
#       (best_validation_loss * 100., best_iter + 1, test_score * 100.))
# print(('The code for file ' +
#        os.path.split(__file__)[1] +
#        ' ran for %.2fm' % ((end_time - start_time) / 60.)), file=sys.stderr)


... training


NameError: name 'n_valid_batches' is not defined